## Libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from collections.abc import Iterable

## Custom Plots

In [2]:
def custom_subplots(*args, facecolor='#FFFFFFFF', size=None, **kwargs):
    """
    Create a custom subplot with specified facecolor and size.

    Parameters:
    - *args: Positional arguments for plt.subplots.
    - facecolor: Background color of the subplot.
    - size: Tuple specifying the size of the figure (width, height).
    - **kwargs: Additional keyword arguments for plt.subplots.

    Returns:
    - fig: The created figure.
    - axes: The created axes.
    """

    # Set full-width default if size is not given
    if size is None:
        size = (16, 4)

    fig, axes = plt.subplots(*args, **kwargs)
    fig.set_size_inches(size)

    if not (isinstance(axes, Iterable) and not isinstance(axes, np.ndarray) or (isinstance(axes, np.ndarray) and axes.ndim > 0)):
        axes = [axes]
      
    for ax in np.ravel(axes):
        ax.set_xticks([])
        ax.set_yticks([])

        for spine in ax.spines.values():
            spine.set_edgecolor('black')
            spine.set_linewidth(1)

    plt.subplots_adjust(wspace=0.05)
    fig.patch.set_facecolor(facecolor)

    if len(np.ravel(axes)) == 1:
        axes = np.ravel(axes)[0]
    else:
        axes = np.ravel(axes)

    return fig, axes